# Téléchargement des données forex avec un Webscrapper
## Objectif
L'objectif de ce notebook est de permettre de récupérer l'historiques des données du forex dans le temps. Pour cela, nous utiliserons un webscrapper à l'aide de Selenium pour récupérer tout ce qui se trouve sur le site internet www.histdata.com qui met à disposition des données en libre téléchargement. 

## Un Webscrapper ? 

Nous utiliserons un webscrapper puisque les données sont disponibles mais sont difficiles d'accès. Du moins, l'historique des paires des devises est indépendante à chaque paire et chaque année. Donc avec environ 50 paires et 20 ans on peut s'attendre à répéter un bon millier de fois le même processus. D'où l'intérêt d'automatiser cela à l'aide d'un web scrapper. 

**Dans les prochains notebook nous reformaterons les données, ici, il ne s'agit que de les telécharger, ranger dans les bons dossiers puis les extraires des .zip**

## Import des librairies

Avant toute chose, il est important d'avoir installé selenium. 

```
pip install selenium
```

Ensuite, en fonction de votre navigateur télécharger le driver associé. Pour ma part, je suis sur chrome dont le lien de téléchargement est https://chromedriver.chromium.org/downloads. 
Extraire le driver si nécessaire et mettez le à la racine du notebook. 
Tout est fin prêt. 

In [1]:
import os  # for path dirs and files
import time  # to not to saturate the website 
import glob  # to recover the files after downloading them
import shutil  # to move the files
import zipfile  # to unzip files at the very end of the process


# to automatize the scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Ouverture du driver

In [2]:
path_driver = os.path.join('.', 'chromedriver.exe')
driver = webdriver.Chrome(path_driver)

<ipython-input-2-5c24f24948a3>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path_driver)


## Récupération des paires de devise

Comme indiqué plus haut, on veut automatiser la récupération de l'historique des différentes paires de devise. Donc pour cela, il faut savoir quelles sont les devises alors on utilise selenium pour aller sur la page de listant les devises dont l'historique sous format ASCII des ticks M1 est présent. 

**Acceptez les cookies**


In [3]:
# open the webpage on which we can list all the pairs of currency
driver.get("https://www.histdata.com/download-free-forex-data/?/ascii/1-minute-bar-quotes")
print(driver.title)

Download Free Forex Data – HistData.com


In [4]:
# recover all the pairs of currency from the table from the HTML page
html_pair_table = driver.find_element_by_css_selector("#content > div > table > tbody")
html_pairs = html_pair_table.find_elements_by_tag_name("strong")

# isolate the text from the HTML
pairs = [pair.text for pair in html_pairs]

print(f"Found {len(pairs)} pairs:")
', '.join(pairs)

C:\Users\athevenot\Anaconda3\lib\site-packages\selenium-4.0.0a7-py3.8.egg\selenium\webdriver\remote\webdriver.py:691: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\athevenot\Anaconda3\lib\site-packages\selenium-4.0.0a7-py3.8.egg\selenium\webdriver\remote\webelement.py:365: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


Found 66 pairs:


'EUR/USD, EUR/CHF, EUR/GBP, EUR/JPY, EUR/AUD, USD/CAD, USD/CHF, USD/JPY, USD/MXN, GBP/CHF, GBP/JPY, GBP/USD, AUD/JPY, AUD/USD, CHF/JPY, NZD/JPY, NZD/USD, XAU/USD, EUR/CAD, AUD/CAD, CAD/JPY, EUR/NZD, GRX/EUR, NZD/CAD, SGD/JPY, USD/HKD, USD/NOK, USD/TRY, XAU/AUD, AUD/CHF, AUX/AUD, EUR/HUF, EUR/PLN, FRX/EUR, HKX/HKD, NZD/CHF, SPX/USD, USD/HUF, USD/PLN, USD/ZAR, XAU/CHF, ZAR/JPY, BCO/USD, ETX/EUR, EUR/CZK, EUR/SEK, GBP/AUD, GBP/NZD, JPX/JPY, UDX/USD, USD/CZK, USD/SEK, WTI/USD, XAU/EUR, AUD/NZD, CAD/CHF, EUR/DKK, EUR/NOK, EUR/TRY, GBP/CAD, NSX/USD, UKX/GBP, USD/DKK, USD/SGD, XAG/USD, XAU/GBP'

## Téléchargement de l'historique disponible

Nous avons donc la liste de paire de devises. Un rapide coup d'oeil et on remarque que les années disponibles se situent entre 2000 et 2020 à l'heure où j'écris ces lignes. Donc on navigue année par année, paire par paire pour tout télécharger. 

**J'ai ajouté plusieurs time.sleep() pour ne pas surcharger le siteweb. Le webscrapping n'est pas illégal mais limite alors il est préférable de respecter le site et de laisser du temps entre les différentes reqûetes. Cette cellule est longue donc tu peux aller te prendre un café ou deux (dure environ 3 heures)**

In [5]:
base_url = 'https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes'
years = range(2000, 2020 + 1)

# for each pair and year, download the historical data and status report file
for i, pair in enumerate(pairs):
    print(f'Downloading {pair} pair from {years[0]} to {years[-1]} [{i+1}/{len(pairs)}]...')

    
    for year in years:
        time.sleep(1)
        
        # get the url corresponding to (pair, year)
        pair = pair.replace('/', '')
        suffix = f'{pair.lower()}/{year}'
        url = f'{base_url}/{suffix}'
        
        # go to the right url
        driver.get(url)
        
        # click to download the historical data if exists
        try:
            hist_data_button = driver.find_element_by_xpath(f"//a[@id='a_file']")
            hist_data_button.click()
            time.sleep(3)
        except:
            pass
        
        # click to download the status report if exists
        try:
            status_file = driver.find_element_by_xpath(f"//a[@id='a_status']")
            status_file.click()
            time.sleep(3)
        except:
            pass

## Rangement des fichiers téléchargées. 

Lors de notre téléchargement nous avons téléchargé tous le fichiers des paires de devise et par année. Pour chacun de ces couples on a un fichier de report de status lors de problèmes et un fichier avec l'historique des données. 
Tous ces fichiers sont dans le dossier téléchargement. Ce que l'on veut c'est pouvoir regreouper tous les fichiers dans des dossiers par paire de devise.


Par sécurité, on créer alors les dossiers suivants s'il n'existent pas encore:

```
.data/
|
|__EURUSD/
|  |
|  |__histdata/
|  |
|  |__status/
|
|__EURUSD/
|  |
|  |__histdata/
|  |
|  |__status/
|
...


```

In [6]:
# root data folder
data_dir = os.path.join('.', 'data')

# directories to create if they not already exist
directories_to_ensure = [data_dir]

for pair in pairs:
    pair = pair.replace('/', '')
    # root pair folder and its historical folder and status folder
    pair_dir = os.path.join(data_dir, pair)
    pair_dir_histdata = os.path.join(pair_dir, 'histdata')
    pair_dir_status = os.path.join(pair_dir, 'status')
    
    # append these 3 directories of this pair to the dir to ensure
    directories_to_ensure.append(pair_dir)
    directories_to_ensure.append(pair_dir_histdata)
    directories_to_ensure.append(pair_dir_status)

# now create every directory listed if it does not exist yet
for directory in directories_to_ensure:
    if not os.path.exists(directory):
        os.makedirs(directory)

Et maintenant qu'on est sûr que les dossiers existent pour chaque paire de devise alors on peut déplacer tous nos fichiers du dossier téléchargements aux dossiers des paires respectives. 

In [7]:
download_root = r'C:/Users/athevenot/Downloads'

dl_files = glob.glob(os.path.join(download_root, 'HISTDATA*'))

for file in dl_files:
    *_, filename = file.split(os.sep)
    filename_without_extention, extension = filename.split('.')
    
    *_, pair, tick_year = filename_without_extention.split('_')
    
    file_data = 'histdata' if extension == 'zip' else 'status'
    
    new_dir = os.path.join('.', 'data', pair, file_data)
    shutil.move(file, new_dir)

## Vérification des téléchargement
Pour vérifier que les téléchargements se sont bien déroulés on va effectuer une vérification simple. Le but étant pour chaque paire de devise, de vérifier que les années des fichiers se suivent. 

Si ce n'est pas le cas on pourra corriger cela en effectuant les opérations précédentes de façon manuelle.

On vérifie aussi que le nombre de fichier ne soit pas égal à 0.

In [8]:
# for each folder of each pair 
# we check the year of the files are following
global_error = False

for pair in pairs:
    pair = pair.replace('/', '')
    for folder in ['histdata', 'status']:
        
        # get all the years in a list from files
        files = glob.glob(os.path.join('.', 'data', pair, folder, '*'))

        # check there at least one file
        if not len(files):
            print(f'There is no file for the {pair} pair in {folder}:')
            global_error = True

        
        
        filenames = [f.split(os.sep)[-1].split('.')[0] for f in files]
        years = [int(f[-4:]) for f in filenames]
        
        # sort the years ascendingly
        years = list(sorted(years))
                
        # catch if there is a gap between year
        error = False
        for y_0, y_1 in zip(years[:-1], years[1:]):
            if y_1 - y_0 != 1:
                error = True
                global_error = True
                
        # display years of the given pair and folder in case of an error
        if error:
            print(f'There is an error for the {pair} pair in {folder}:')
            print(years)

        
global_error = ' at least one' * global_error
print(f'There is{global_error} global error')
    

There is global error


## Extraction des données

On peut maintenant extraire tous les fichiers zip des différents dossiers (pas besoin de les supprimer ça ne prend pas beaucoup de place mais on pourrait en effet le faire)

In [9]:
for pair in pairs:
    pair = pair.replace('/', '')
    target_folder = os.path.join('.', 'data', pair, 'histdata')
    # get all the years in a list from files
    files_to_unzip = glob.glob(os.path.join(target_folder, '*.zip')) 

    for file in files_to_unzip:

        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(target_folder)
